In [ ]:
!pip install PyWavelets pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 46.3 MB/s eta 0:00:00


In [ ]:
import io
import os
import pywt
import random
import librosa
import scipy.signal
import numpy as np
import pandas as pd
import soundfile as sf
from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# 1. Time Stretch
def time_stretching(segment):
    rate = random.uniform(0.96, 1.06)  # 4% slower to 6% faster
    stretched = librosa.effects.time_stretch(segment.astype(float), rate=rate)
    return stretched.astype(np.float32)

# 2. Pitch Shift
def pitch_shifting(segment, sr):
    s = sr
    r = random.uniform(-1.2, 1.2)  # -1.2 to 1.2 semitones
    shifted = librosa.effects.pitch_shift(segment.astype(float), sr=s, n_steps=r)
    return shifted.astype(np.float32)

# 3. Reverberation
def reverberation(segment, sr):
    room_size = random.uniform(0.4, 0.55)  # Moderate reverb
    damping = random.uniform(0.35, 0.5)  # Damping factor
    impulse = np.random.normal(0, 1, int(sr * 0.25))  # 0.25s impulse
    impulse = scipy.signal.lfilter([1], [1, -damping], impulse)
    reverb = np.convolve(segment.astype(float), impulse, mode='same') * room_size
    reverb = reverb / (np.max(np.abs(reverb)) + 1e-8)  # Normalize to prevent clipping
    return reverb.astype(np.float32)

# 4. Micro Time Warping
def micro_time_warping(segment):
    warp_factor = random.uniform(1.02, 1.05)  # 2–5% stretch/compress
    indices = np.linspace(0, len(segment) - 1, int(len(segment) * warp_factor))
    warped = np.interp(indices, np.arange(len(segment)), segment.astype(float))
    return warped.astype(np.float32)

# 5. Phase Shift
def phase_shift(segment, sr):
    fft_data = np.fft.fft(segment.astype(float))
    freqs = np.fft.fftfreq(len(segment), 1/sr)
    magnitude = np.abs(fft_data)
    phase = np.angle(fft_data)
    phase_shift_deg = random.uniform(30, 90)  # 30° to 90° shift
    phase_shift_rad = np.deg2rad(phase_shift_deg)
    mask = (np.abs(freqs) >= 500) & (np.abs(freqs) <= 5000)
    phase[mask] += phase_shift_rad
    fft_data_shifted = magnitude * (np.cos(phase) + 1j * np.sin(phase))
    shifted = np.real(np.fft.ifft(fft_data_shifted))
    return shifted.astype(np.float32)

# 6. Low Bitrate MP3 Compression
def low_bitrate_mp3_compression(segment, sr):
    buffer = io.BytesIO()
    sf.write(buffer, segment.astype(float), sr, format='WAV')
    buffer.seek(0)
    audio = AudioSegment.from_wav(buffer)
    compressed_buffer = io.BytesIO()
    audio.export(compressed_buffer, format="mp3", bitrate="128k")  # 128 kbps
    compressed_buffer.seek(0)
    compressed_audio = AudioSegment.from_mp3(compressed_buffer)
    compressed = np.array(compressed_audio.get_array_of_samples()).astype(np.float32) / 32768.0
    if len(compressed) != len(segment):
        compressed = librosa.resample(compressed, orig_sr=sr, target_sr=sr, res_type='kaiser_best')
        compressed = compressed[:len(segment)]  # Trim/pad to match original length
    return compressed.astype(np.float32)

# 7. Time Stretch + Pitch Shift
def time_stretch_pitch_shift(segment, sr):
    stretched = time_stretching(segment)
    shifted = pitch_shifting(stretched, sr)
    return shifted.astype(np.float32)

# 8. Bandpass Filtering (for Reverberation + Bandpass Filtering)
def bandpass_filtering(segment, sr):
    low_cutoff = random.uniform(200, 400)  # 200–400 Hz
    high_cutoff = random.uniform(7000, min(10000, sr / 2 - 1))  # 12,000–14,000 Hz
    sos = scipy.signal.butter(4, [low_cutoff, high_cutoff], fs=sr, btype='bandpass', output='sos')
    return scipy.signal.sosfilt(sos, segment.astype(float))

# 9. Reverberation + Bandpass Filtering
def reverb_bandpass(segment, sr):
    reverb = reverberation(segment, sr)
    filtered = bandpass_filtering(reverb, sr)
    return filtered.astype(np.float32)

# 10. Time Stretch + Reverberation
def time_stretch_reverb(segment, sr):
    stretched = time_stretching(segment)
    reverb = reverberation(stretched, sr)
    return reverb.astype(np.float32)

def apply_transformations(segment, sr, choice):
    segment = np.array(segment, dtype=np.float32)

    if np.any(np.isnan(segment)) or np.any(np.isinf(segment)):
        segment = np.nan_to_num(segment)
    for choice in choice:
        if choice == "1":
            segment = time_stretching(segment)
        elif choice == "2":
            segment = pitch_shifting(segment, sr)
        elif choice == "3":
            segment = reverberation(segment, sr)
        elif choice == "4":
            segment = micro_time_warping(segment)
        elif choice == "5":
            segment = phase_shift(segment, sr)
        elif choice == "6":
            segment = low_bitrate_mp3_compression(segment, sr)
        elif choice == "7":
            segment = time_stretch_pitch_shift(segment, sr)
        elif choice == "8":
            segment = reverb_bandpass(segment, sr)
        elif choice == "9":
            segment = time_stretch_reverb(segment, sr)
        elif choice == "10":
            pass  # No transformation
        else:
            print(f"Invalid choice: {choice}")
    return segment


***Time Stretching***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/time_stretching_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretching_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shift")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shifting")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shift
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shifting
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 1
Choice chosen: ['1']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)


    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    # Post-processing
    # Convert to Mono
    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    # Resampling
    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    # Normalize the audio
    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/s089_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/cl229_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/f138_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/e091_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/r075_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/h081_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/h105_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/o112_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretching_forged_tracks/e236_forged.wav
Forged audio saved to: /content/drive/MyDrive

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretching_output.csv


***Pitch Shifting***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/pitch_shifting_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/pitch_shifting_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 2
Choice chosen: ['2']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/e194_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/p109_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/e192_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/o194_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/o228_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/c196_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/s221_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/e186_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/pitch_shifting_forged_tracks/h202_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tr

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/pitch_shifting_output.csv


***Reverberation***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/reverberation_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/reverberation_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/reverberation_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 3
Choice chosen: ['3']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/h215_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/o230_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/e223_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/o086_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/r118_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/c090_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/p223_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/h165_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverberation_forged_tracks/cl035_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/rev

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/reverberation_output.csv


***Micro Time Warping***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/micro_time_warping_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/micro_time_warping_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shift")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shift
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 4
Choice chosen: ['4']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/f170_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/p139_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/c016_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/o156_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/c056_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/p020_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/r205_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/r008_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/micro_time_warping_forged_tracks/s234_forged.wav
Forged audio saved 

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/micro_time_warping_output.csv


***Phase Shifting***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/phase_shifting_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/phase_shifting_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 5
Choice chosen: ['5']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/f213_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/o171_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/h144_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/o213_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/e012_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/c033_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/cl039_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/h167_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/phase_shifting_forged_tracks/e150_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_t

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/phase_shifting_output.csv


***Low Bitrate MP3 Compression***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/low_bitrate_mp3_compression_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/low_bitrate_mp3_compression_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 6
Choice chosen: ['6']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/p036_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/h241_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/c150_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/c069_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/f175_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/s150_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/o092_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/low_bitrate_mp3_compression_forged_tracks/r012_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tra

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/low_bitrate_mp3_compression_output.csv


***Time Stretch + Pitch Shift***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/time_stretch_pitch_shift_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretch_pitch_shift_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 7
Choice chosen: ['7']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/j224_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/e231_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/r022_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/h155_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/p205_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/f111_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/h132_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_shift_forged_tracks/h080_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_pitch_s

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretch_pitch_shift_output.csv


***Reverb + Bandpass Filtering***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/reverb_bandpass_filtering_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/reverb_bandpass_filtering_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 8
Choice chosen: ['8']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/p225_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/cl143_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/f225_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/r034_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/e172_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/o210_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/j021_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_bandpass_filtering_forged_tracks/f191_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/reverb_band

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/reverb_bandpass_filtering_output.csv


***Time Stretch + Reverb***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/time_stretch_reverb_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretch_reverb_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 9
Choice chosen: ['9']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/c093_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/cl040_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/p247_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/j186_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/h096_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/c182_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/j148_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/e063_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/time_stretch_reverb_forged_tracks/h182_forged.wav
Forged au

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/time_stretch_reverb_output.csv


***No Transformation***

In [ ]:
input_csv_path = "/content/drive/MyDrive/input_files_for_data_augmentation/no_transformation_input.csv"
output_csv_path = "/content/drive/MyDrive/output_log_files_for_data_augmentation/no_transformation_output.csv"
forged_folder_path = "/content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks"

In [ ]:
df = pd.read_csv(input_csv_path)

print("\nSelect transformation(s) to apply on the segment:")
print("1. Time Stretching")
print("2. Pitch Shifting")
print("3. Reverberation")
print("4. Micro Time Warping")
print("5. Phase Shifting")
print("6. Low Bitrate MP3 Compression")
print("7. Time Stretch + Pitch Shift")
print("8. Reverb + Bandpass Filtering")
print("9. Time Stretch + Reverb")
print("10. No Transformation")

choice = input("Enter transformation by number: ")
print(f"Choice chosen: {choice}")


Select transformation(s) to apply on the segment:
1. Time Stretching
2. Pitch Shifting
3. Reverberation
4. Micro Time Warping
5. Phase Shifting
6. Low Bitrate MP3 Compression
7. Time Stretch + Pitch Shift
8. Reverb + Bandpass Filtering
9. Time Stretch + Reverb
10. No Transformation
Enter transformation by number: 10
Choice chosen: ['10']


In [ ]:
results = []
for index, row in df.iterrows():
    file_path = row["file_path"].strip()
    track_length = row["track_length_seconds"]
    segment_duration = row["segment_duration"]
    start_copy_segment = row["start_copy_segment"]
    end_copy_segment = start_copy_segment + segment_duration

    audio, sr = librosa.load(file_path, sr=None)

    start_sample = int(start_copy_segment * sr)
    end_sample = int(end_copy_segment * sr)
    segment = audio[start_sample:end_sample]

    transformed_segment = apply_transformations(segment, sr, choice)
    modified_segment_duration = len(transformed_segment) / sr

    valid_ranges = [
        (0, start_copy_segment - modified_segment_duration),
        (end_copy_segment, track_length - modified_segment_duration)
    ]
    valid_ranges = [r for r in valid_ranges if r[1] > r[0]]

    if valid_ranges:
        start_paste_segment = random.uniform(valid_ranges[0][0], valid_ranges[0][1]) \
            if len(valid_ranges) == 1 else random.choice([
            random.uniform(valid_ranges[0][0], valid_ranges[0][1]),
            random.uniform(valid_ranges[1][0], valid_ranges[1][1])
        ])
        end_paste_segment = start_paste_segment + modified_segment_duration
    else:
        continue

    paste_sample = int(start_paste_segment * sr)

    new_audio = np.concatenate((audio[:paste_sample], transformed_segment, audio[paste_sample:]))

    if len(new_audio.shape) > 1:
        new_audio = np.mean(new_audio, axis=0)

    TARGET_SR = 16000
    if sr != TARGET_SR:
        audio = librosa.resample(audio, orig_sr=sr, target_sr=TARGET_SR)
        sr = TARGET_SR

    audio = audio / np.max(np.abs(audio))

    forged_file_name = os.path.basename(file_path).rsplit(".", 1)[0] + "_forged.wav"
    forged_file_path = os.path.join(forged_folder_path, forged_file_name)

    sf.write(forged_file_path, new_audio, sr)
    print(f"Forged audio saved to: {forged_file_path}")

    transformations_applied = choice
    results.append([
        file_path, track_length, segment_duration,
        start_copy_segment, end_copy_segment,
        start_paste_segment, end_paste_segment,
        transformations_applied, forged_file_path
    ])

Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/cl133_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/s108_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/c208_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/o081_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/e242_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/s193_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/p240_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/r136_forged.wav
Forged audio saved to: /content/drive/MyDrive/forged_tracks/no_transformation_forged_tracks/c218_forged.wav
Forged audio saved to: /con

In [ ]:
output_df = pd.DataFrame(results, columns=[
    "file_path", "track_length_seconds", "segment_duration",
    "start_copy_segment", "end_copy_segment",
    "start_paste_segment", "end_paste_segment",
    "transformations_applied", "forged_file_path"
])
output_df.to_csv(output_csv_path, index=False)
print(f"\nProcessing complete. Log saved at: {output_csv_path}")


Processing complete. Log saved at: /content/drive/MyDrive/output_log_files_for_data_augmentation/no_transformation_output.csv
